In [ ]:
import pymongo
import os
from dotenv import load_dotenv
import exam_settings
import importlib

#importlib.reload(exam_settings)

<module 'exam_settings' from '/Users/sang/Documents/Source/Python/python_webscrap/cert_exam/exam_settings/__init__.py'>

In [61]:
load_dotenv() 

db_client = pymongo.MongoClient(os.environ['DB_URI'])
db = db_client['db_certificates']
collection = db['tb_aws_cloud_practitioner']


In [66]:
#get question(s) and export to images
def get_n_export_question_2_img(question_num):
    pipeline = [
                {"$match": {'exported': 0, 'type': 'multiple-choice'}},
                {"$sample": {"size": question_num}} #randomly
            ]
    random_documents = list(collection.aggregate(pipeline))
    index = 1
    for doc in random_documents:
        str_index = str(index) + ') '
        #question first
        html_question = '<div class="question">'+str_index + doc['question']+'</div>'
        html_answers_start = '<div class="answers">'
        html_answers_end = '</div>'
        #First image: options without explanations
        for key in doc['options'].keys():
            html_answers_start += f'''
                <div class="answer">
                    <label>{key}. {doc['options'][key]}</label>
                </div>
                <div class="explanation">
                    <label>{key}. {doc['explanation'][key]}</label>
                </div>'''
        #1 doc 1 image
        exam_settings.generate_image(exam_settings.html_head_str + html_question + html_answers_start + html_answers_end + exam_settings.html_tail_str, doc['uuid'] + '.png')
        #Second image: options with explanations
        html_answers_start = '<div class="answers">'
        for key in doc['options'].keys():
            correct_class = ''
            if doc['answer'] == key:
                correct_class = ' correct'
            html_answers_start += f'''
                <div class="answer">
                    <label class="{correct_class}">{key}. {doc['options'][key]}</label>
                </div>
                <div class="explanation show">
                    <label>{doc['explanation'][key]}</label>
                </div>'''
        exam_settings.generate_image(exam_settings.html_head_str + html_question + html_answers_start + html_answers_end + exam_settings.html_tail_str, doc['uuid'] + '_explain.png')
        #
        index += 1
    # print(random_documents)

#test
#get_n_export_question_2_img(5)